In [1]:
# Import relevant packages
import tensorflow as tf
import numpy as np
import h5py
import scipy.io
import skimage.io 

In [2]:
# Load data and set up sets
tf.reset_default_graph()
f = h5py.File('data/data_all.h5', 'r')

X_train = f['train_images'].value
y_train = f['train_labels'].value

X_val = f['validation_images'].value
y_val = f['validation_labels'].value

/usr/local/lib/python3.5/dist-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [12]:
# Define a U-net model
def make_unet():
    inputs = tf.keras.layers.Input(shape=(512, 192, 3))
    
    conv1 = tf.keras.layers.Conv2D(32, 3, activation = 'relu', padding = 'same')(inputs)
    conv1 = tf.keras.layers.Conv2D(32, 3, activation = 'relu', padding = 'same')(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same')(pool1)
    conv2 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same')(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(pool2)
    conv3 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(conv3)
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same')(pool3)
    conv4 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same')(conv4)
    drop4 = tf.keras.layers.Dropout(0.5)(conv4)
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same')(pool4)
    conv5 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same')(conv5)
    drop5 = tf.keras.layers.Dropout(0.5)(conv5)

    up6 = tf.keras.layers.Conv2D(256, 2, activation = 'relu', padding = 'same')(tf.keras.layers.UpSampling2D(size = (2,2))(drop5))
    merge6 = tf.keras.layers.concatenate([conv4,up6], axis = 3)
    conv6 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same')(merge6)
    conv6 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same')(conv6)

    up7 = tf.keras.layers.Conv2D(128, 2, activation = 'relu', padding = 'same')(tf.keras.layers.UpSampling2D(size = (2,2))(conv6))
    merge7 = tf.keras.layers.concatenate([conv3,up7], axis = 3)
    conv7 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(merge7)
    conv7 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(conv7)

    up8 = tf.keras.layers.Conv2D(64, 2, activation = 'relu', padding = 'same')(tf.keras.layers.UpSampling2D(size = (2,2))(conv7))
    merge8 = tf.keras.layers.concatenate([conv2,up8], axis = 3)
    conv8 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same')(merge8)
    conv8 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same')(conv8)

    up9 = tf.keras.layers.Conv2D(32, 2, activation = 'relu', padding = 'same')(tf.keras.layers.UpSampling2D(size = (2,2))(conv8))
    merge9 = tf.keras.layers.concatenate([conv1,up9], axis = 3)
    conv9 = tf.keras.layers.Conv2D(32, 3, activation = 'relu', padding = 'same')(merge9)
    conv9 = tf.keras.layers.Conv2D(32, 3, activation = 'relu', padding = 'same')(conv9)
    conv9 = tf.keras.layers.Conv2D(2, 3, activation = 'relu', padding = 'same')(conv9)
    conv10 = tf.keras.layers.Conv2D(1, 1, activation = 'sigmoid')(conv9)
    conv10 = tf.keras.layers.Reshape((conv10.shape[1], conv10.shape[2]))(conv10)
    model = tf.keras.Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = tf.train.AdamOptimizer(0.00001), loss = 'mean_squared_error')
    
    model.summary()

    return model

model = make_unet()
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='logs/{}'.format('model_name'))
checkpoint = tf.keras.callbacks.ModelCheckpoint('./model.{epoch:02d}-{val_loss:.10f}.hdf5', monitor='val_loss', verbose=0, period=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 512, 192, 3)  0                                            
__________________________________________________________________________________________________
conv2d_96 (Conv2D)              (None, 512, 192, 32) 896         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_97 (Conv2D)              (None, 512, 192, 32) 9248        conv2d_96[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_16 (MaxPooling2D) (None, 256, 96, 32)  0           conv2d_97[0][0]                  
__________________________________________________________________________________________________
conv2d_98 

In [9]:
hist = model.fit(X_train, y_train,
          batch_size=10,
          epochs=1000,
          verbose=1,
          validation_data=(X_val, y_val),
          callbacks=[early_stop, tensorboard, checkpoint])

Train on 490 samples, validate on 10 samples
Epoch 1/1000
490/490 [==============================] - 111s 226ms/sample - loss: 114.7720 - val_loss: 113.2372
Epoch 2/1000
490/490 [==============================] - 110s 224ms/sample - loss: 113.4841 - val_loss: 113.1696
Epoch 3/1000
490/490 [==============================] - 110s 224ms/sample - loss: 113.4308 - val_loss: 113.0994
Epoch 4/1000
490/490 [==============================] - 111s 226ms/sample - loss: 113.3459 - val_loss: 112.9935
Epoch 5/1000
490/490 [==============================] - 111s 226ms/sample - loss: 113.2352 - val_loss: 112.8749
Epoch 6/1000
170/490 [=========>....................] - ETA: 1:11 - loss: 115.0614

KeyboardInterrupt: 

In [46]:
tf.keras.models.save_model(model,'model.h5')